Getting Data using CURL
-----------------------

We now move into a more interesting topic: How to get data from Internet sources. For that, we will use a command-line tool of Unix, called `curl`. (Later in class, we will learn how to achieve the same using Python, but for quick testing, curl is often the standard method used.) We will also use a tool called `jq` to interact with JSON output. (Do not worry, we will revisit both these later in class.)

_Often, curl and jq do not come preinstalled, so the first time that we use them, we need to issue the appropriate command for installing it.  To install it, simply type:_

In [ ]:
!sudo apt-get -y install curl
!sudo apt-get -y install jq

Let's start by retrieving a simple text file, which we will use later in the class, to illustrate how different shell commands work. The sample data file is hosted online. You can use terminal commands to copy this remote file. Simply type:

In [ ]:
!curl -L 'https://dl.dropboxusercontent.com/u/16006464/IPDS/sample.txt'

The columns in this tab-separated data correspond to [order id] [time of order] [user id] [ordered item], something similar to what might be encountered in practice. If you wish, you can copy-paste the data written above into a text editor, making sure there is a newline following each of the ordered item columns (the columns with alphabetic characters).

To store the output to a file, we also add the `-o [output file]` in the command. (We are also going to see in the next session how to use _output redirection_ to store the output to a file.)

In [ ]:
!curl -L 'https://dl.dropboxusercontent.com/u/16006464/IPDS/sample.txt' -o data/sample.txt

In [ ]:
!ls data/

This will pull the file to the directory `/home/ubuntu/data/`, creating a new file called `sample.txt`. If we do not want to see any statistics about the download, we can use the `-s` option:

In [ ]:
!curl  -s -L 'https://dl.dropboxusercontent.com/u/16006464/IPDS/sample.txt' -o data/sample.txt

Now, let's try to use curl to get access to some real data. A key component of today's data ecosystem is the existence of `Web APIs` which provide functionality for a variety of tasks.

#### Where am I?

For example, let's try to figure out programmatically the location of the computer where the iPython server is running. We can access the API call by issuing the following command:



In [1]:
!curl -s "http://freegeoip.net/json/" | jq .

{
  "ip": "54.174.159.22",
  "country_code": "US",
  "country_name": "United States",
  "region_code": "VA",
  "region_name": "Virginia",
  "city": "Ashburn",
  "zip_code": "20149",
  "time_zone": "America/New_York",
  "latitude": 39.0481,
  "longitude": -77.4729,
  "metro_code": 511
}


While this does not look nice to a human, for a computer is a perfectly legitimate answer. This format is called "JSON", and is an efficient and very commonly used way to trasfer data today on the Internet.
| jq controls presentation

Now, let's examine a few more web APIs, just for fun:

#### What's the weather?

Now, let's use the OpenWeather API to get the weather details in our location. (The details of the API calls are available at http://openweathermap.org/api.)

In [2]:
!curl -s "http://api.openweathermap.org/data/2.5/weather?\
&appid=ffb7b9808e07c9135bdcc7d1e867253d\
&q=New%20York,NY,USA\
&units=imperial\
&mode=json" | jq .

{
  "coord": {
    "lon": -74.01,
    "lat": 40.71
  },
  "weather": [
    {
      "id": 500,
      "main": "Rain",
      "description": "light rain",
      "icon": "10d"
    }
  ],
  "base": "stations",
  "main": {
    "temp": 39.59,
    "pressure": 1015.94,
    "humidity": 94,
    "temp_min": 39.59,
    "temp_max": 39.59,
    "sea_level": 1019.42,
    "grnd_level": 1015.94
  },
  "wind": {
    "speed": 22.41,
    "deg": 69.5014
  },
  "rain": {
    "3h": 0.76
  },
  "clouds": {
    "all": 100
  },
  "dt": 1485195496,
  "sys": {
    "message": 0.1904,
    "country": "US",
    "sunrise": 1485173566,
    "sunset": 1485209038
  },
  "id": 5128581,
  "name": "New York",
  "cod": 200
}


You will notice that we asked the service to return to us the data in JSON format. For that API, we can also ask the data to be returned in a different format, called XML, which is wordlier. (We will get back to these formats later in the semester.)

In [3]:
!curl -s "http://api.openweathermap.org/data/2.5/weather?\
&q=New%20York,NY,USA\
&units=imperial\
&mode=xml\
&appid=ffb7b9808e07c9135bdcc7d1e867253d"

<current><city id="5128581" name="New York"><coord lon="-74.01" lat="40.71"></coord><country>US</country><sun rise="2017-01-23T12:12:46" set="2017-01-23T22:03:58"></sun></city><temperature value="39.59" min="39.59" max="39.59" unit="fahrenheit"></temperature><humidity value="94" unit="%"></humidity><pressure value="1015.94" unit="hPa"></pressure><wind><speed value="22.41" name="Severe Gale"></speed><gusts></gusts><direction value="69.5014" code="ENE" name="East-northeast"></direction></wind><clouds value="100" name="overcast clouds"></clouds><visibility></visibility><precipitation value="0.76" mode="rain" unit="3h"></precipitation><weather number="500" value="light rain" icon="10d"></weather><lastupdate value="2017-01-23T18:18:16"></lastupdate></current>

#### What's the sentiment?

Now let's try to use a web service to automatically analyze the sentiment for a piece of text. (The service comes from the [IBM's Alchemy API](http://www.alchemyapi.com/api/sentiment/textc.html#textsentiment))

In [4]:
!curl -s "http://access.alchemyapi.com/calls/text/TextGetTextSentiment" \
-d "outputMode=json" \
-d "apikey=4b46c7859a7be311b6f9389b12504e302cac0a55" \
-d "text=I did not dislike it. " | jq .

{
  "status": "OK",
  "usage": "By accessing AlchemyAPI or using information generated by AlchemyAPI, you are agreeing to be bound by the AlchemyAPI Terms of Use: http://www.alchemyapi.com/company/terms.html",
  "totalTransactions": "1",
  "language": "english",
  "docSentiment": {
    "type": "neutral"
  }
}


#### And a few synonyms

And now just a demo of a web API that I created myself a few years back. It analyzes Wikipedia to figure out different ways that people use to refer to the same entity



In [6]:
!curl -s "http://wikisynonyms.ipeirotis.com/api/Donald_Trump" | jq .

{
  "http": 200,
  "message": "success",
  "terms": [
    {
      "term": "Donald Trump",
      "canonical": 1,
      "oskill": 0
    },
    {
      "term": "Donald John Trump",
      "canonical": 0,
      "oskill": 0
    },
    {
      "term": "The Donald",
      "canonical": 0,
      "oskill": 0
    },
    {
      "term": "Donald J. Trump",
      "canonical": 0,
      "oskill": 0
    },
    {
      "term": "Donald trump",
      "canonical": 0,
      "oskill": 0
    },
    {
      "term": "Trump Timepieces",
      "canonical": 0,
      "oskill": 0
    },
    {
      "term": "Donald J. Trump Signature Collection",
      "canonical": 0,
      "oskill": 0
    },
    {
      "term": "Donald J Trump Men's Collection",
      "canonical": 0,
      "oskill": 0
    },
    {
      "term": "Donald J. Trump Men's Collection",
      "canonical": 0,
      "oskill": 0
    },
    {
      "term": "Donald John Trump, Sr",
      "canonical": 0,
      "oskill": 0
    },
    {
      "term": "Donald John T

## Exercise

The following websites contain listing of many useful APIs

* https://www.mashape.com 
* http://www.programmableweb.com/
* http://www.mashery.com/
* http://apigee.com/ 

Mashape is my own personal favorite in terms of user-friendliness and also has examples directly expressed using CURL. but the others are pretty nice as well. Your task: search through these websites and find a web API that does something that you like. Use CURL to issue a web API call to this service. 